In [ ]:
# Pobranie bibliotek oraz modelu
!pip install transformers torch fastapi uvicorn nest-asyncio
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name="gpt2"


tokenizer = GPT2Tokenizer.from_pretrained(model_name, resume_download=True)
model = GPT2LMHeadModel.from_pretrained(model_name, resume_download=True)
model.eval()


if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
from fastapi import FastAPI #framework do tworzenia REST api (m.in. obsługuje żądania HTTP) | FastAPI tworzy aplikacje serwerową
import nest_asyncio #pozwala uruchomić asynchroniczny kod w środowisku np. Jupyter
import uvicorn #Serwer ASGI (Asynchronous Server Gateway Interface), czyli serwer, który obsługuje FastAPI. To on nasłuchuje na porcie i odpowiada na żądania HTTP.
import asyncio

nest_asyncio.apply()  # potrzebne w Jupyterze

app = FastAPI()

@app.get("/generate")
def generate_text(prompt: str, max_length: int = 50):
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_length=max_length, do_sample=True)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"generated_text": text}

config=uvicorn.Config(app=app,host="127.0.0.1",port=5000,log_level="info")
server=uvicorn.Server(config=config)

asyncio.get_event_loop().run_until_complete(server.serve())


INFO:     Started server process [9004]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:63564 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:63564 - "GET /favicon.ico HTTP/1.1" 404 Not Found


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     127.0.0.1:63567 - "GET /generate?prompt=Hej HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9004]


KeyboardInterrupt: 

In [ ]:
words = ["sztuczna", "inteligencja", "przyszłość"]

# Tokeny GPT-2
tokens = [tokenizer.encode(w, add_special_tokens=False) for w in words]
print("Tokeny:", tokens)

# Embeddingi dla tokenów
embeddings = model.transformer.wte.weight  # wte = word token embeddings
for i, w_tokens in enumerate(tokens):
    for t in w_tokens:
        print(f"Słowo: {words[i]}, Token: {t}, Embedding shape: {embeddings[t].shape}")


In [ ]:
prompt = "Sztuczna inteligencja"
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Forward pass, bez generacji
with torch.no_grad():
    outputs = model(inputs)
    logits = outputs.logits  # shape: [1, seq_len, vocab_size]

# Softmax dla ostatniego tokena
import torch.nn.functional as F
last_token_logits = logits[0, -1, :]
probs = F.softmax(last_token_logits, dim=-1)

# 100 najbardziej prawdopodobnych tokenów
top_k = torch.topk(probs, 100)
for token_id, prob in zip(top_k.indices.tolist(), top_k.values.tolist()):
    print(tokenizer.decode([token_id]), prob)


In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(100), top_k.values.cpu().numpy())
plt.title("Top 100 prawdopodobnych tokenów")
plt.show()
